# Clickbait Spoiler Generation using BERT

In [1]:
# This is necessary to fix the imports
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../src')))

In [2]:
MODEL_CHECKPOINT = "distilbert-base-uncased-distilled-squad"

In [3]:
BATCH_SIZE = 3
N_EPOCHS = 5
LEARNING_RATE = 2e-5
SAVE_CHECKPOINT_PATH = "../src/models/bert/clickbait"

In [4]:
from utils.bert import MODEL_CHECKPOINT, TOKENIZER
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

/Users/ddsantos/miniconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ddsantos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ddsantos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ddsantos/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-uncased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("../data/parsed/bert/clickbait_data.py", data_files={
    "train": "train.jsonl", 
    "test": "test.jsonl", 
    "validation": "validation.jsonl"
})

Using custom data configuration default-c436ff332249223c


Dataset clickbait_data downloaded and prepared to /Users/ddsantos/.cache/huggingface/datasets/clickbait_data/default-c436ff332249223c/0.0.0/264c21381577df7a68b078d3233557c5c51acba80b028ef1987ae9ae6d00581a. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 394.96it/s]


In [6]:
import utils.bert

train_dataset = raw_datasets["train"].map(
    utils.bert.preprocess_training,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

100%|██████████| 4/4 [00:05<00:00,  1.46s/ba]


(3200, 9056)

In [7]:
test_dataset = raw_datasets["test"].map(
    utils.bert.preprocess_validation,
    batched=True,
    remove_columns=raw_datasets["test"].column_names,
)
len(raw_datasets["test"]), len(test_dataset)

100%|██████████| 1/1 [00:01<00:00,  1.02s/ba]


(602, 1721)

In [8]:
validation_dataset = raw_datasets["validation"].map(
    utils.bert.preprocess_validation,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

100%|██████████| 1/1 [00:00<00:00,  1.18ba/s]


(198, 623)

In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")
tf_train_dataset = model.prepare_tf_dataset(
    train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [10]:
from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(tf_train_dataset) * N_EPOCHS
optimizer, schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [11]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath="../src/models/bert/checkpoints/model.{epoch:02d}_{accuracy:.2f}-{loss:.2f}_{val_accuracy:.2f}-{val_loss:.2f}.h5"),
    tf.keras.callbacks.TensorBoard(log_dir="../src/models/bert/logs")
]

In [12]:
print(tf.config.list_physical_devices("GPU"))
history = model.fit(
    tf_train_dataset,
    callbacks=callbacks,
    epochs=N_EPOCHS,
    verbose=1
)

2023-05-24 05:12:15.175857: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Epoch 1/5


In [ ]:
model.save_pretrained(SAVE_CHECKPOINT_PATH)

In [ ]:
tf_test_dataset = model.prepare_tf_dataset(
    test_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [ ]:
predictions = model.predict(tf_test_dataset)

In [ ]:
utils.bert.compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    validation_dataset,
    raw_datasets["validation"],
)

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model=SAVE_CHECKPOINT_PATH, tokenizer=TOKENIZER)

context = """
At some point toward the end of every job interview, the hiring manager will likely turn the tables and ask, \"Do you have any questions for me?\"\nThis is the time to ask smart, thoughtful questions — perhaps your final opportunity to assess whether the job would be a good fit, and your final chance to impress the hiring manager.\nBusiness Insider recently spoke with Becca Brown, cofounder of Solemates, a brand of women's shoe-care products, who knows a thing or two about interviewing.\nBefore launching her own business, Brown, who has a bachelor's from Harvard University and an MBA from Columbia, spent a lot of time interviewing job candidates at Goldman Sachs, where she held various roles, including analyst, wealth adviser, and chief of staff.\nShe was also part of the investment bank's Harvard recruiting team, she says.\n\"I interviewed anywhere from 20 to 30 job candidates a year, so in total, I interviewed over 100 people at Goldman Sachs,\" she tells Business Insider.\nShe says that candidates asked her some impressive questions — like \"What's the most challenging part of your job?\" and \"What's one of the most interesting projects you've worked on?\" — but there was one question she always hoped she'd be asked, but almost never was: \"Where do you see yourself in five years?\"\n\"I like this question — and yet no one ever asked it — because it's difficult to answer,\" she says. \"It's an important question for anyone to be asking him or herself, and so if ever a candidate were to ask this question, it would have stood out.\"\nShe continues:\nI think this is a good question for interviewees to ask because, as a candidate, if you see where the person interviewing you is headed, you can decide if that trajectory is in line with your career objectives. While they don't have to be completely correlated, it's helpful for the candidate to have some indication of the interviewer's direction.\nGet the latest Goldman Sachs stock price here.
"""
question="A woman who interviewed over 100 people at Goldman Sachs says there's one question she always hoped candidates would ask her, but they never did"
expected = "\"Where do you see yourself in five years?\""

In [ ]:
prediction = question_answerer(question=question, context=context)
prediction

In [ ]:
import evaluate
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")
# bertscore = evaluate.load("bertscore")

In [ ]:
meteor_results = meteor.compute(predictions=[prediction["answer"]], references=[expected])
bleu_results = bleu.compute(predictions=[prediction["answer"]], references=[expected])
# bertscore_results = bertscore.compute(predictions=prediction, references=expected, lang="en")

In [ ]:
print(f"{question};\n\nExpected Spoiler: {expected};\n\nSpoiler Predicted: {prediction['answer']};\n\n###\n\n")
print(f"Meteor: {meteor_results['meteor']}\nBLEU-4: {bleu_results['bleu']}")
# print(f"Meteor: {meteor_results['meteor']}\nBLEU-4: {bleu_results['bleu']}\nBERTscore Mean F1: {sum(bertscore_results['f1'])/len(bertscore_results['f1'])}")